In [ ]:

# It seems that search engines, although gladly offering to perform image searches on your browser,
# fancy much less offering this service to Python scripts.

# This script uses ddgs to get urls of images
# I've been using this script to generate the resources on my site. (example: https://ehudmwork.github.io/resources/images/cats.json)
# It runs successfully on Kaggle.com
#NB: Kaggle requires (simple and free) process to use internet on scripts.

# How to use :% substitute /cats/search-term
#              set max_results to wish (currently it doesn't give more than 100 results) 
!pip install ddgs==9.5.5  > /dev/null
import ddgs

import ddgs 
from ddgs import DDGS

with DDGS() as ddgs:
    results = ddgs.images("cats", max_results=500)


import json
json.dumps(results)